In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Recommendation Engine\n",
    "This notebook generates product recommendations using Collaborative Filtering + Content-Based Filtering."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.metrics.pairwise import cosine_similarity"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Preprocessed Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = pd.read_csv('clean_retail_data.csv')\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Collaborative Filtering (User-Item Matrix)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "user_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', fill_value=0)\n",
    "user_item_matrix.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "similarity_matrix = cosine_similarity(user_item_matrix)\n",
    "similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)\n",
    "similarity_df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Recommendation Function"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def recommend_products(customer_id, n=5):\n",
    "    if customer_id not in similarity_df.index:\n",
    "        return []\n",
    "    similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:6]\n",
    "    similar_users = user_item_matrix.loc[similar_customers.index]\n",
    "    recommendations = similar_users.sum().sort_values(ascending=False).head(n)\n",
    "    return recommendations.index.tolist()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "recommend_products(customer_id=user_item_matrix.index[0], n=5)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Export Recommendation Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "results = []\n",
    "\n",
    "for user in user_item_matrix.index:\n",
    "    recs = recommend_products(user, n=5)\n",
    "    results.append({\n",
    "        'CustomerID': user,\n",
    "        'RecommendedProducts': recs\n",
    "    })\n",
    "\n",
    "results_df = pd.DataFrame(results)\n",
    "results_df.to_csv('model_results.csv', index=False)\n",
    "results_df.head()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
